# TP 2 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
# import Pkg; 
# Pkg.add("GraphRecipes"); Pkg.add("Plots");
# using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

In [3]:
function separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left

    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end

    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible

    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop
end

exploreNextNode_depthfirst! (generic function with 1 method)

In [4]:

#Définition structure de donnée
#------------------------------
#           MODÈLE
#------------------------------




mutable struct Model
    price::Vector{Int64}
    weight::Vector{Int64}
    capacity::Int64
    x::Vector{Float64}
    value::Int64
    Model(price,weight,capacity,x,value) = new(price,weight,capacity,x,value)
    Base.copy(s::Model) = Model(copy(s.price), copy(s.weight),copy(s.capacity),copy(s.x),copy(s.value))
end

#Créer un modele
function createModel_LP(price::Vector{Int64},weight::Vector{Int64},capacity::Int64)
    return Model(price,weight,capacity,zeros(Float64,length(weight)),0)
end

#------------------------------
#           CALCUL BORNE
#------------------------------

#Calcul la borne supérieure 1 ou 2 d'un problème de sac à dos
#   Input:
#      model: le modèle actuel
#      mode: borne à calculer (vaut "1" ou "2")
#   
#   Output:
#      valB : valeur de la borne 
#      index : The position of the object having the best ratio


function calculBorne(model::Model,mode::String="1")
    if mode == "1"
        
        #Création des ratios
        r = map((x,y) -> x./y,model.price,model.weight)

        #Obtention du max des ratios
        max_ri,index = findmax(r)

        #Calcul borne sup
        borneSup = (model.capacity)*max_ri

        #Renvoie fonction
        return borneSup,index
    else

        # Trier les objets par ratio décroissant
        r = map((x, y) -> x / y, model.price, model.weight)
        sorted_indices = sortperm(r, rev=true)
        
        # Initialiser la capacité restante et la borne supérieure
        remaining_capacity = model.capacity
        borneSup = 0.0

        # Parcourir les objets triés
        for i in sorted_indices
            if model.weight[i] <= remaining_capacity
                # Ajouter l'objet entier si possible
                borneSup += model.price[i]
                remaining_capacity -= model.weight[i]
            else
                # Ajouter la fraction de l'objet qui rentre dans le sac
                fraction = remaining_capacity / model.weight[i]
                borneSup += fraction * model.price[i]
                break
            end
        end

        return borneSup, sorted_indices[1]
        
    end
    

end


#------------------------------------------
#           TEST DE SONDABILITÉ
#------------------------------------------

function testTA(model::Model)
    return model.capacity < 0
end

function testTO(upperbound::Float64,bestSol::Float64)
    return upperbound < bestSol
end

function testTR(xi::Float64)
    return xi != 0 && xi != 1
end

function print_model(model::Model)
    println("-------------------------")
    println("model : ")
    println("price :" , model.price)
    println("weight :" , model.weight)
    println("capacity :" , model.capacity)
    println("x :" , model.x)
    println("value :" , model.value)
    println("-------------------------")
end

# Recherche et supprime un element du sac tout en actualisant les x_i
function suppressFromBagAndUpdateX!(elemToDelIndex::Int64,model::Model,initListWeight::Vector{Int64},initListPrice::Vector{Int64},xValue::Float64)
        #Element à supprimer
        elemToDelweight = model.weight[elemToDelIndex]
        elemToDelprice = model.price[elemToDelIndex]
        
        #Vérifier la correspondance entre la liste restante dans model et la liste de base de notre premier appel
        

        indicesWeight = findall(x -> x == elemToDelweight ,initListWeight)
        indicesPrice = findall(x -> x == elemToDelweight ,initListWeight)
        indices = intersect(indicesPrice,indicesWeight) # On prend l'intersection des indices potentiel de prix et de poids

        if length(indices) > 1 
            # println("ERREUR")
        end

        reelIndex = indices[1] #On accede à l'indice dans la liste de base
        model.x[reelIndex] = xValue #On modifie la valeur du xi

        #On supprime l'objet du sac
        deleteat!(model.weight, elemToDelIndex)
        deleteat!(model.price, elemToDelIndex)
        
end     



suppressFromBagAndUpdateX! (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [5]:



function solveKnapInstance(filename, mode="1")

    price, weight, capacity = readKnaptxtInstance(filename)

    modelToSolve = createModel_LP(price, weight, capacity)

    sol::Model = Model([], [], 0, [], 0.0)
    BestProfit::Float64 = -1.0

    function solveNode(model::Model)
        # DEBUG
        # print_model(model)

        # Calculer la borne avec l'algo de borne (calculBorne)
        upperbound, index = calculBorne(model, mode)
        upperbound = model.value + upperbound
        # println("Upperbound : ",upperbound)

        #  2) Effectuer les tests TA,TO,TR
        if (testTA(model) || testTO(upperbound, BestProfit) || testTR(model.x[index]))
            #On fait rien ce n'est pas une solution
            
            
            #if (testTA(model))
            #    println("Oups... TA")
            #elseif (testTO(upperbound, BestProfit))
            #    println("Oups... TO")
            #else
            #    println("Oups... TR")
            #end 
            
        else


            #On crée le fils gauche
            childLeft = copy(model)

            # Paramétrage du fils gauche
            childLeft.capacity = childLeft.capacity - childLeft.weight[index]
            childLeft.value = childLeft.value + childLeft.price[index]
            suppressFromBagAndUpdateX!(index, childLeft, weight, price, 1.0)
            println("update model left child")
            print_model(childLeft)

            #On crée le fils droit
            childRight = copy(model)
            suppressFromBagAndUpdateX!(index, childRight, weight, price, 0.0)
            println("update model right child")
            print_model(childRight)

            # Passer au prochain noeud


            #  On compare la BS de chaque variable et on execute celui dont la BS est la plus grande
            if isempty(childLeft.weight) && isempty(childLeft.price)  #Dans le cas où l'on a une solution valable
                if model.value > BestProfit #Si cette solution est meilleure que celle(s) déjà trouvée(s)
                    BestProfit = model.value
                    sol = copy(model)
                    println("Solution enregistrée !")

                end
            else
                #Calul de la borne du fils gauche
                upperboundLeft,_ = calculBorne(childLeft)
                upperboundLeft = upperboundLeft + childLeft.value

                if isempty(childRight.weight) && isempty(childRight.price)  #Dans le cas où l'on a une solution valable
                    if model.value > BestProfit #Si cette solution est meilleure que celle(s) déjà trouvée(s)
                        BestProfit = model.value
                        sol = model
                        println("Solution enregistrée !")
                    end
                else
                    #Calul de la borne du fils droit
                    upperboundRight,_ = calculBorne(childRight)
                    upperboundRight = upperboundRight + childRight.value

                    # On branche sur la variable qui a la plus grande borne supérieure
                    if upperboundLeft > upperboundRight
                        #println("Selected Left Child")
                        #println("Upperbound Left Child : ",upperboundLeft)
                        #println("Upperbound Right Child : ",upperboundRight)
                        solveNode(childLeft)
                        solveNode(childRight)
                    else
                        #println("Selected Right Child")
                        #println("Upperbound Left Child : ",upperboundLeft)
                        #println("Upperbound Right Child : ",upperboundRight)
                        solveNode(childRight)
                        solveNode(childLeft)
                    end
                end
            end
        end


    end
    solveNode(modelToSolve)
    println("DONE")
    return BestProfit, sol
end

solveKnapInstance (generic function with 2 methods)

### Affichage du résultat final

In [6]:
function solveNdisplayKnap(filename, mode="1")

    println("\n Branch-and-Bound for solving a knapsack problem. \n\n Solving instance '" * filename * "'\n")

    BestProfit, Bestsol = solveKnapInstance(filename, mode)
    # println("\n******\n\nOptimal value = ",BestProfit,"\n with configuration:")
    # print_model(Bestsol)

end

solveNdisplayKnap (generic function with 2 methods)

In [7]:
import Pkg; Pkg.add("Glob")
using Glob

function processFolder(folderPath)

    txtFiles = glob("*.txt", folderPath)
    stats = Vector{Tuple{Float64, Float64}}()

    for file in txtFiles
        println("Processing file: $file")        
        time_taken_1 = @elapsed solveNdisplayKnap(file, "1")
        println("Borne 1 : ", time_taken_1, " seconds")
        time_taken_2 = @elapsed solveNdisplayKnap(file, "2")
        println("Borne 2 : ", time_taken_2, " seconds")

        push!(stats, (time_taken_1, time_taken_2))
    end

    return stats
end
    
function ecartRelatif(x,y)
    return abs(x-y)/x
end

function processStats(stats)
    avg_ecart = 0.0
    for stat in stats
        avg_ecart += ecartRelatif(stat[1], stat[2])
    end
    avg_ecart /= length(stats)
    println("Average relative difference between bounds: ", avg_ecart)
end

FOLDER_PATH = "Instances/"
stats = processFolder(FOLDER_PATH)
processStats(stats)

# println("press enter to exit ! ")
# readline()

   Resolving package versions...


    Updating `~/Mir/GitRepos/RO/Rendu/Project.toml`
  [c27321d9] + Glob v1.3.1
    Updating `~/Mir/GitRepos/RO/Rendu/Manifest.toml`
  [c27321d9] + Glob v1.3.1


Processing file: Instances/knapPI_11_20_1000_1_-1428.opb.txt

 Branch-and-Bound for solving a knapsack problem. 

 Solving instance 'Instances/knapPI_11_20_1000_1_-1428.opb.txt'



update model left child
-------------------------
model : 
price :

[114, 38, 133, 95, 171, 918, 408, 714, 510, 114, 76, 57, 204, 1020, 816, 190, 510, 114, 714]
weight :[582, 194, 679, 485, 873, 594, 264, 462, 330, 582, 388, 291, 132, 660, 528, 970, 330, 582, 462]
capacity :574
x :

...

, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :970
-------------------------
update model left child
-------------------------
model : 
price :[364, 260, 104, 520, 416, 260, 364]
weight :[462, 330, 132, 660, 528, 330, 462]
capacity :-264
x :[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :1178
-------------------------
update model right child
-------------------------
model : 
price :[364, 260, 104, 520, 416, 260, 364]
weight :[462, 330, 132, 660, 528, 330, 462]
capacity :0
x :[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :970
-------------------------
update model left child
-------------------------
model : 
price :[260, 104, 520, 416, 260, 364]
weight :[330, 132, 660, 528, 330, 462]
capacity :-462
x :[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :1334
-------------------------
update mod

582
-------------------------
update model left child
-------------------------
model : 
price :[485, 312, 873, 468, 208, 364, 260, 582, 388, 291, 104, 520, 416, 970, 260, 582, 364]
weight :[485, 396, 873, 594, 264, 462, 330, 582, 388, 291, 132, 660, 528, 970, 330, 582, 462]
capacity :-291
x :[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :1261
-------------------------
update model right child
-------------------------
model : 
price :[485, 312, 873, 468, 208, 364, 260, 582, 388, 291, 104, 520, 416, 970, 260, 582, 364]
weight :[485, 396, 873, 594, 264, 462, 330, 582, 388, 291, 132, 660, 528, 970, 330, 582, 462]
capacity :388
x :[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :582
-------------------------
update model left child
-------------------------
model : 
price :[312, 873, 468, 208, 364, 260, 582, 388, 291, 104, 520, 416, 970, 260, 582, 364]
weight :[396, 873,

Excessive output truncated after 524290 bytes.

, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :1164
-------------------------
update model right child
-------------------------
model : 
price :[312, 468, 208, 364, 260, 388, 291, 104, 520, 416, 970, 260, 582, 364]
weight :[396, 594, 264, 462, 330, 388, 291, 132, 660, 528, 970, 330, 582, 462]
capacity :388
x :[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :582
-------------------------
update model left child
-------------------------
model : 
price :[312, 468, 208, 364, 260, 291, 104, 520, 416, 970, 260, 582, 364]
weight :[396, 594, 264, 462, 330, 291, 132, 660, 528, 970, 330, 582, 462]
capacity :0
x :[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value :970
-------------------------
update model right child
-------------------------
model : 
price :[312, 468, 208, 364, 260, 291, 104, 520, 416, 970, 260, 582, 364]
weight :[39

InterruptException: InterruptException:

# Questions préliminaires

1. À chaque nœud, le solveur va résoudre un problème d'optimisation sous contraintes. Chaque valeur de \(x(i)\) est a priori binaire et indique si oui ou non l'objet est inclus dans le sac. Cependant, le solveur peut trouver une solution comportant un ou plusieurs \(x(i)\) dont la valeur n'est pas binaire mais flottante. C'est sur ce critère que l'algorithme se base pour créer les branches à partir d'un nœud. Lorsqu'un résultat obtenu par un solveur sur un nœud de profondeur \(i\) comporte une ou plusieurs valeurs non booléennes (notons les \(x_{\lambda1}, x_{\lambda2}, ..., x_{\lambda m})\) dans l'ordre lexicographique, l'algorithme sélectionne \(x_{\lambda_1}\) car c'est le premier dans l'ordre lexicographique, et il crée deux nœuds de profondeur \(i+1\) et une arête possède la condition \(x_{\lambda_1} = 1\) et l'autre possède la condition \(x_{\lambda_1} = 0\).

2. L'algorithme calcule la borne supérieure et inférieure de la manière suivante:

   - Borne supérieure:
     \[ \forall i \in [1..n], \text{BorneSup}(x[i]) = \min(x[i],1) \]

   - Borne inférieure:
     \[ \forall i \in [1..n], \text{BorneInf}(x[i]) = \max(x[i],0) \]

3. L'algorithme calcule la TA, la TO et la TR de la manière suivante:

   - TA: Le problème d'optimisation sous contraintes ne possède pas de solution.
   - TO: Le problème d'optimisation sous contraintes possède une solution moins intéressante que celle déjà enregistrée.
   - TR: Le problème d'optimisation sous contraintes possède une solution plus intéressante que celle déjà enregistrée.

4. La stratégie de parcours est un parcours en profondeur de l'arbre créé en prenant d'abord le fils gauche plutôt que le fils droit.


# Code et analyse

## Détail de l'implémentation

Pour l'implémentation de l'algorithme de Branch and Bounds, nous avons opté pour une fonction utilisant un principe récursif.

### Borne supérieure

```julia
function calculBorne(model::Model,mode::String="1")
    if mode == "1"
        % Creation des ratios
        r = map((x,y) -> x./y,model.price,model.weight)

        # Obtention du max des ratios
        max_ri,index = findmax(r)

        # Calcul borne sup
        borneSup = (model.capacity)*max_ri

        # Renvoie fonction
        return borneSup,index
    else
        # Trier les objets par ratio décroissant
        r = map((x, y) -> x / y, model.price, model.weight)
        sorted_indices = sortperm(r, rev=true)
        
        # Initialiser la capacité restante et la borne supérieure
        remaining_capacity = model.capacity
        borneSup = 0.0

        # Parcourir les objets triés
        for i in sorted_indices
            if model.weight[i] <= remaining_capacity
                # Ajouter l'objet entier si possible
                borneSup += model.price[i]
                remaining_capacity -= model.weight[i]
            else
                # Ajouter la fraction de l'objet
                fraction = remaining_capacity / model.weight[i]
                borneSup += fraction * model.price[i]
                break
            end
        end

        return borneSup, sorted_indices[1]
    end
end
```
Pour le calcul de la borne 1, nous avons simplement calculé le ratio de chaque objet et nous avons pris le maximum de ces ratios pour le multiplier par la capacité du sac. Nous avons ensuite retourné la borne supérieure ainsi que l'indice de l'objet de ratio maximum. Pour la borne 2, nous avons trié les objets par ratio décroissant et nous avons ajouté les objets tant que la capacité du sac le permettait. Si la capacité du sac ne permettait pas d'ajouter un objet entier, nous avons ajouté une fraction de cet objet. Nous avons ensuite retourné la borne supérieure ainsi que l'indice de l'objet qui a été ajouté en dernier.

Règle de séparation
Dans notre implémentation, la règle de séparation est basée sur l'indice retourné par la fonction calculant la borne supérieure. Cet indice représente l'objet avec le meilleur ratio (ou le dernier ajouté pour la borne 2) et sert à déterminer le point de branchement. Deux branches sont générées : dans la première, l'objet est inclus dans le sac à dos (fils gauche), tandis que dans la seconde, l'objet n'est pas inclus (fils droit). Cette approche permet une exploration efficace de l'espace des solutions en se focalisant sur les objets les plus prometteurs.

## Tests de sondabilité (TA, TO, TR)
```julia 
function testTA(model::Model)
    return model.capacity < 0
end

function testTO(upperbound::Float64,bestSol::Float64)
    return upperbound < bestSol
end

function testTR(xi::Float64)
    return xi != 0 && xi != 1
end
 ```

 - **TA:** Si le poids total des objets sélectionnés est supérieur à la capacité du sac, la branche est non sondable.
- **TO:** Si la borne supérieure est inférieure à la meilleure solution trouvée, la branche est également non sondable.
- **TR:** Si la décision de prendre ou non l'objet est déjà prise, la branche est redondante et donc non sondable.

## Stratégie d'exploration

Notre stratégie d'exploration utilise une approche gloutonne basée sur les bornes supérieures calculées. Après la génération des branches, nous sélectionnons d'abord la branche avec la borne supérieure la plus élevée pour l'exploration. Cette méthode permet de prioriser les chemins les plus prometteurs, augmentant ainsi les chances de trouver rapidement une solution optimale.

## Structure de données

La structure de données clé de notre implémentation est la structure `Model`. Elle encapsule toutes les informations nécessaires pour représenter un nœud dans l'arbre de recherche du Branch-and-Bound :

```julia
mutable struct Model
    price::Vector{Int64}
    weight::Vector{Int64}
    capacity::Int64
    x::Vector{Float64}
    value::Int64
end
```

- **price** et **weight** sont des vecteurs d'entiers représentant respectivement les valeurs et les poids des objets. Ces vecteurs sont essentiels pour le calcul des bornes et la détermination des objets à inclure ou à exclure lors de la création des branches. 
- **capacity** est un entier représentant la capacité restante du sac à dos à un nœud donné. Elle est cruciale pour les tests de sondabilité, en particulier le test TA.
- **x** est un vecteur de nombres flottants représentant la solution partielle à un nœud donné, où chaque élément indique si un objet spécifique est inclus (1.0) ou exclu (0.0) du sac.
- **value** est un entier indiquant la valeur totale des objets inclus dans le sac à un nœud donné. C'est un indicateur clé de la performance d'une solution partielle.

## Stockage des informations

Pour stocker les informations nécessaires à la résolution du problème, nous avons utilisé différents objets de stockage:

- Les variables
- La récursivité
- Une structure de données

En effet, voici les variables qui nous permettent de stocker les données au cours du problème:

- **price::Vector{Int64}** & **weight::Vector{Int64}** : ces variables servent à stocker les prix et les poids du problème initial. En effet, au cours du problème, la structure de donnée `model` varie et perd l'indexation initial des objets. Ces deux listes nous servent à les retrouver.
- **sol::Model**: cette variable stocke le modèle qui a donné la meilleure solution.
- **BestProfit::Int64**: Cette variable stocke le meilleur prix trouvé jusqu'ici pour que le test TR puisse avoir accès à la plus grande valeur du sac trouvée.

La récursivité permet de stocker le modèle au cours des différentes itérations. Enfin, la structure de donnée `model` encapsule toutes les données propres à un modèle comme expliqué précédemment.

# Améliorations possibles

Voici les améliorations que nous pourrions implémenter:

- La méthode de résolution de la relaxation pour la borne 2 est naïve en $O(n)$ avec $n$ la taille de la liste des objets. On peut optimiser cela en
- Notre règle d'exploration parcourt les nœuds par priorité de borne supérieure et parmi les nœuds à parcourir, on les parcourt selon l'ordre décroissant de profondeur de l'arbre. C'est une caractéristique propre au fait que notre programme résout récursivement le problème. Pour changer cela, il faudrait mettre en place une file, ce qui est moins naturel dans notre cas.
- Notre algorithme utilise une structure de données, ce qui est coûteux en mémoire. On pourrait économiser du temps de calcul et de la mémoire en modifiant notre algorithme afin d'en retirer la structure de données.


